In [2]:
# Import libaries and illustris library
import numpy as np
import illustris_python as il
import matplotlib.pyplot as plt
%matplotlib inline
import random
import subroutines.distance as df

# everything in units of h! (h= 0.7)

In [5]:
# load data of citical points and segments from DisPerSE output
crits_file = '/Users/alex/Documents/Research/Astro/vm_shared_folder_v2/il_1_snap_135_posdata_halo_mass_cutoff.txt.NDnet_c5.07e-07.up.NDskl.a.crits'
crits = np.loadtxt(crits_file, comments='#')[:,0:5]
crits_max = crits[crits[:,4]>2][:,0:3]/0.7
crits_pos = crits_max

segs_file = '/Users/alex/Documents/Research/Astro/vm_shared_folder_v2/il_1_snap_135_posdata_halo_mass_cutoff.txt.NDnet_s2.02.up.NDskl.BRK.S1000.a.segs'
segs_pos = np.loadtxt(segs_file, comments='#')[:,0:3]/0.7

halo_file_1 = "/Users/alex/Documents/Research/Astro/vm_shared_folder_v2/il_1_snap_135_posdata_halo_mass_cutoff.txt"
halo_positions = np.loadtxt(halo_file_1, comments='#', delimiter=',')[:,0:3]/0.7
halo_mass = np.loadtxt(halo_file_1, comments='#', delimiter=',')[:,3]

In [6]:
# load file with basic information:
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda

galaxy_file = './data/illustris_data/galaxyData_zeph_Il1_snap135.txt'
galaxies = np.genfromtxt(galaxy_file, comments='#')

In [9]:
# filter galaxy data through mask and compute distance
# putting everything in units of h! (the inital file is) (h= 0.7)

#index_pos_data = 5
galaxies_positions = galaxies[:,5:8]

In [19]:
galaxy_segs_distance_mat = df.distance_function(galaxies_positions, segs_pos)

distance_function done with 0.0 %
distance_function done with 10.0 %
distance_function done with 20.0 %
distance_function done with 30.0 %
distance_function done with 40.0 %
distance_function done with 50.0 %
distance_function done with 60.0 %
distance_function done with 70.0 %
distance_function done with 80.0 %
distance_function done with 90.0 %
distance_function done with 100.0 %


In [20]:
galaxy_crits_distance_mat = df.distance_function(galaxies_positions, crits_pos)

distance_function done with 0.0 %
distance_function done with 10.0 %
distance_function done with 20.0 %
distance_function done with 30.0 %
distance_function done with 40.0 %
distance_function done with 50.0 %
distance_function done with 60.0 %
distance_function done with 70.0 %
distance_function done with 80.0 %
distance_function done with 90.0 %
distance_function done with 100.0 %


In [7]:
# Add the output of the distance function to the end of the matrix
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda 18: Index of segment/critical point, 19:absolute distance to filament

galaxy_crits_joined = np.concatenate(( galaxies[:,:], galaxy_crits_distance_mat[:,:]),axis=1)

galaxy_segs_joined = np.concatenate(( galaxies[:,:], galaxy_segs_distance_mat[:,:]), axis=1)

NameError: name 'galaxy_crits_distance_mat' is not defined

In [8]:
# create matrix for align function input
gal_align_segs = df.align_function(galaxy_segs_joined[:,11:14], galaxy_segs_joined[:,18:20], segs_pos)

# Add: 20:cos(theta) 21:sin(theta) 22:(theta)
galaxy_segs_data = np.concatenate((galaxy_segs_joined[:,:], gal_align_segs[:,2:5]), axis=1)

NameError: name 'galaxy_segs_joined' is not defined

In [62]:
# 0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x,y,z
# 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 
# 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda 18: Index of nearest segment, 19:absolute distance to filament, 
# 20:cos(theta) 21:sin(theta) 22:(theta) 23-25: nearest segement coordinates 26: Index of nearest critical point, 
# 27: absolute distanceto crit., 28-30: nearest critical point coordinates, 31: bool in cluster, 32: bool in filament
galaxy_data = np.zeros(((galaxy_segs_data.shape[0]), 33))

for i in range (0,galaxy_segs_data.shape[0]):
    galaxy_data[i,0:23] = galaxy_segs_data[i,:]
    galaxy_data[i,23:26] = segs_pos[int(galaxy_data[i,18]),:]
    galaxy_data[i,26:28] = galaxy_crits_distance_mat[i,:]
    galaxy_data[i,28:31] = crits_pos[int(galaxy_data[i,26]),:]

galaxy_data_crits_mask = (galaxy_data[:,27] < 2000)
galaxy_data_segs_mask = (galaxy_data[:,19] < 700)
    
for i in range (0, galaxy_segs_data.shape[0]):
    if (galaxy_data_crits_mask[i] == True):
        galaxy_data[i,31] = 1
        galaxy_data[i,32] = 0
    elif(galaxy_data_segs_mask[i] == True):
        galaxy_data[i,31] = 0
        galaxy_data[i,32] = 1
    else:
        galaxy_data[i,31] = 0
        galaxy_data[i,32] = 0

In [63]:
np.savetxt('./data/galaxy_data_full_sim_v3.txt', galaxy_data, delimiter =' ', 
           header = '0:HaloNumber, 1:SubhaloNumber, 2:StellarMass, 3:GasMass, 4:DarkMatterMass, 5-7:SubhaloPosition x, SubhaloPosition y, SubhaloPosition z, 8-10:CoM Velocity x, CoM Velocity y, CoM Velocity z, 11-13:AngularMomentum x, AngularMomentum y, AngularMomentum z, 14:Kappa, 15:Flatness, 16:Ellip, 17:Lambda, 18: Index of nearest segment, 19:absolute distance from galaxy to nearest segment point 20:dot product of angular momentum vector with segment vector 21-23: nearest segement coordinates 24: Index of nearest critical point, 25: absolute distance from galaxy to nearest critical point 26-28: nearest critical point coordinates, 29: bool in cluster, 30: bool in filament'
           )

In [38]:
# Now do same thing for halo data:

# 0-2: CM position, 3: halo mass, 4: index of nearest filament point, 5: filament absolute distance
# 6: index of nearest crit point, 7: crit absolute distance, 8: bool in cluster, 9: bool in filament

halo_data = np.zeros((halo_positions.shape[0], 10))
halo_data[:,0:3] = halo_positions
halo_data[:,3] = halo_mass
halo_data = halo_data[halo_data[:,3]>=0.5][:,:]

In [39]:
halo_segs_distance_mat = df.distance_function(halo_data[halo_data[:,3]>=0.5][:,0:3], segs_pos)

distance_function done with 0.0 %
distance_function done with 10.0 %
distance_function done with 20.0 %
distance_function done with 30.0 %
distance_function done with 40.0 %
distance_function done with 50.0 %
distance_function done with 60.0 %
distance_function done with 70.0 %
distance_function done with 80.0 %
distance_function done with 90.0 %
distance_function done with 100.0 %


In [40]:
halo_crits_distance_mat = df.distance_function(halo_data[halo_data[:,3]>=0.5][:,0:3], crits_pos)

distance_function done with 0.0 %
distance_function done with 10.0 %
distance_function done with 20.0 %
distance_function done with 30.0 %
distance_function done with 40.0 %
distance_function done with 50.0 %
distance_function done with 60.0 %
distance_function done with 70.0 %
distance_function done with 80.0 %
distance_function done with 90.0 %
distance_function done with 100.0 %


In [41]:
halo_data[:,4:6] = halo_segs_distance_mat
halo_data[:,6:8] = halo_crits_distance_mat

halo_data_crits_mask = (halo_data[:,7] < 2000)
halo_data_segs_mask = (halo_data[:,5] < 700)
    
for i in range (0, halo_data.shape[0]):
    if (halo_data_crits_mask[i] == True):
        halo_data[i,8] = 1
        halo_data[i,9] = 0
    elif(halo_data_segs_mask[i] == True):
        halo_data[i,8] = 0
        halo_data[i,9] = 1
    else:
        halo_data[i,8] = 0
        halo_data[i,9] = 0


In [42]:
np.savetxt('./data/halo_data_full_sim_v2.txt',
           halo_data, header="0-2: CM position, 3: halo mass, 4: index of nearest filament point, 5: filament absolute distance, 6: index of nearest crit point, 7: crit absolute distance, 8: bool in cluster, 9: bool in filament")